# Other Modeling using News dataset

In [ ]:
conh_api='mrCPBin4KFMnfaAsCrHueZtGGDhSxJcr6MxSRVrS'

In [ ]:
# Import the required modules
import cohere
import numpy as np
import pandas as pd
import os,sys
sys.path.append('../scr')
#import config
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import nltk
#nltk.download()

In [ ]:
data = pd.read_csv(r"C:\Users\Genet Shanko\Prompt-Engineering-In-context-learning-with-GPT-3-and-LLMs\Data\Example_data.xlsx - Sheet1.csv")
data.head()
               

In [ ]:
data.info()

In [ ]:
# Obtaining Additional Stopwords From nltk
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
import gensim
from nltk.corpus import stopwords
from gensim.models import Word2Vec
# Removing Stopwords And Remove Words With 2 Or Less Characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stop_words:
            result.append(token)

    return result

In [ ]:
# Import libraries
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
# Applying The Function To The Dataframe and Lower casing and removing punctuations
# Lemmatization
df['Body'] = df['Body'].astype(str).apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['Body'] = df['Body'].astype(str).apply(preprocess)
#data_df['clean'] =" ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(df) if w not in string.punctuation]))
#Summary of title
#lemmatization of Description 
df['Description'] = df['Description'].astype(str).apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['Description'] = df['Description'].astype(str).apply(preprocess)

In [ ]:
# Writing a function to lemmatize words
lmtzr = WordNetLemmatizer()
def lem(text):
    return [lmtzr.lemmatize(word) for word in text]

# Applying the function to each row of the text
# i.e. reducing each word to its lemma
tokens_des = df['Description'].apply(lem)
tokens_body = df['Body'].apply(lem)

In [ ]:
tokens_des

In [ ]:
tokens_body

In [ ]:
lengths = df['Body'].apply(lambda x: len(x)) 
lengths

In [ ]:
lengths = df['Description'].apply(lambda x: len(x)) 
lengths

In [ ]:
rank__to_10 = df['Analyst_Average_Score'].apply(lambda x: 'low' if x < 5 else 'high')
df['rank_1'] = rank__to_10
df['rank_1']

In [ ]:
#add low and high
def handle_sub_class(value):
    if value >= 0 and value < 1:
        return "low_1"
    elif value >= 1 and value < 2:
        return "low_2"
    elif value >= 2 and value < 3:
        return "low_3"
    elif value >= 3 and value < 4:
        return "low_4"
    elif value >= 4 and value < 5:
        return "low_5"
    elif value >= 5 and value < 6:
        return "high_1"
    elif value >= 6 and value < 7:
        return "high_2"
    elif value >= 7 and value < 8:
        return "high_3"
    elif value >= 8 and value < 9:
        return "high_4"
    else:
        return "high_5"

In [ ]:
#rank the average
df['rank_2'] = df['Analyst_Average_Score'].apply(lambda x: handle_sub_class(x))
df['rank_3'] = df['Analyst_Average_Score'].apply(lambda x: handle_sub_class(int("{:.2f}".format(x)[2])))
df['rank_4'] = df['Analyst_Average_Score'].apply(lambda x: handle_sub_class(int("{:.2f}".format(x)[3])))

In [ ]:
df['final_score'] = df['rank_1'].map(str)+'__'+df['rank_2'].map(str)+'__'+df['rank_3'].map(str)+'__'+df['rank_4'].map(str)
df['final_score']

In [ ]:
X,y = df['Title'], df['final_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2, random_state=21)

In [ ]:
intents = y_train.unique().tolist()
print(intents)

## Few-shot classification with the Classify endpoint

###### Few-shot here means we just need to supply a few examples per class and have a decent classifier working. With Cohere’s Classify endpoint, the ‘training’ dataset is referred to as examples. The minimum number of examples per class is five, where each example consists of a text (in our case, the query), and a label (in our case, the label)

In [ ]:
# Set the number of examples per category
#EX_PER_CAT = 6

# Create list of examples containing texts and labels - sample from the dataset
ex_texts, ex_labels = [], []
for intent in intents:
  ex_texts += X_train.tolist()
  ex_labels += y_train.tolist()

print(f'Number of classes: {len(intents)}')
print(f'Total number of examples: {len(ex_texts)}')

###### Get classifications via the Classify endpoint

In [ ]:
# Collate the examples via the Example module
from cohere.classify import Example

examples = list()
for txt, lbl in zip(ex_texts,ex_labels):
  examples.append(Example(txt,lbl))

In [ ]:
# Perform classification
def classify_text(text,examples):
  classifications = co.classify(
    model='medium', # model version - medium-22020720
    inputs=[text],
    examples=examples
    )
  return classifications.classifications[0].prediction

In [ ]:
# Generate classification predictions on the test dataset (this will take a few minutes)
y_pred = X_test.apply(classify_text, args=(examples,)).tolist()

In [ ]:
# Compute metrics on the test dataset
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

### Build your own classifier with the Embed endpoint

##### In this section, we’ll look at how we can use the Embed endpoint to build a classifier. We are going to build a classification model using these embeddings as inputs. For this, we’ll use the Support Vector Machine (SVM) algorithm.

###### Generate embeddings for the input text

In [ ]:
# Get embeddings
def embed_text(text):
  output = co.embed(
                model='medium', # model version - medium-22020720
                texts=text)
  return output.embeddings

# Embed and prepare the inputs
X_train_emb = np.array(embed_text(X_train.tolist()))
X_test_emb = np.array(embed_text(X_test.tolist()))

###### Get classifications via the SVM algorithm

In [ ]:
# Import modules
from sklearn.svm import SVC
from sklearn import preprocessing

# Prepare the labels
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_le = le.transform(y_train)
y_test_le = le.transform(y_test)

# Initialize the model
svm_classifier = SVC(class_weight='balanced')

# Fit the training dataset to the model
svm_classifier.fit(X_train_emb, y_train_le)

In [ ]:
# Generate classification predictions on the test dataset
y_pred_le = svm_classifier.predict(X_test_emb)
y_pred_le

In [ ]:
# Compute metrics on the test dataset
accuracy = accuracy_score(y_test_le, y_pred_le)
f1 = f1_score(y_test_le, y_pred_le, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

##### Finetuning a model

In [ ]:
# Download the training dataset for finetuning
df_train = pd.concat([X_train, y_train],axis=1)
df_train.to_csv("../data/news_finetune.csv", index=False)

#### Create a finetuned model and Get classifications via the Classify endpoint

In [ ]:
# Perform classification using the finetuned model
def classify_text_finetune(text):
  classifications = co.classify(
    model='', # replace with your own finetune model ID 
    inputs=text
    )
  return classifications.classifications

In [ ]:
# Generate classification predictions on the test dataset (this will take a few minutes)
y_pred_raw = classify_text_finetune(X_test.tolist())
y_pred = [y.prediction for y in y_pred_raw]

In [ ]:
# Compute metrics on the test dataset
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

##### it is percived that  how the different options compare performance-wise. And crucially, what’s important to note is the level of control that you have when working with the Classify endpoint.

# 